In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
# data=pd.read_csv("/content/drive/My Drive/hackathon/recipe_table.csv")

In [0]:
import json
file = '/content/drive/My Drive/hackathon/recipe_combos.json'
with open(file) as train_file:
    dict_train = json.load(train_file)
new_data = pd.DataFrame(dict_train)

In [40]:
new_data.head(10)

,items,junkness,preferance,recipe_name,recipe_number
0,"[cocoa, milk, sugar]",1,0,Chocolate Milkshake,1
1,"[salt, wheat flour, butter, cheese]",2,0,Cheese Sticks,2
2,"[vegetable oil, potatoes, wheat flour, salt]",2,1,Potato Crisps,3
3,"[peanuts, sugar, milk, soy lecithin]",0,0,Peanut Cluster,4
4,"[onions, potatoes, carrots, salt, vegetable oil]",0,1,Mixed Vegetables,5
5,"[squid, salt, vegetable oil]",1,0,Fried Squid,6
6,"[bread, butter, cheese, cucumber, tomato]",1,0,Grilled Sandwich,7
7,"[rice, salt, vegetable oil, chicken, pepper]",2,0,Chicken Biriyani,8
8,"[tomatoes, salt, pepper, water, basil]",0,1,Tomato Soup,9
9,"[potatoes, salt, pepper, vegetable oil]",2,1,Potato Wedges,10


In [0]:
all_items_2d = new_data['items'].tolist()
all_items = list(set([item for items in all_items_2d for item in items]))

In [0]:
classes = {'junkness':['Healthy','Moderate','Junk'],'preferance':['non vegan','vegan']}

In [0]:
class_data ={}
for key in classes:
    value =classes[key]
    class_data[key]={}
    for ind,label in enumerate(value):
        class_data[key][label] = new_data[new_data[key]==ind].recipe_number.tolist()

In [0]:
import random
def make_user_history(user_info,num_entries):
    user_history_recipes= []
    user_history_items = []
    length = len(user_info)
    for key in user_info:
        user_pref = user_info[key]
        user_pref_string = classes[key][user_pref]
        
        user_history_recipes += random.sample(class_data[key][user_pref_string],num_entries)
    for recipe in user_history_recipes:
        user_history_items.append(new_data[new_data.recipe_number == recipe]['items'].iloc[0][0])
    return user_history_items

In [17]:
# users = [{'junkness':0,'preferance':1},
#          {'junkness':2,'preferance':0},
#          {'junkness':0,'preferance':2}]
# user = {'a':users[0],'b':users[1],'c':users[2]}
user_list = [
    {
        "junkness":1,
        "preferance":0
    },
    {
        "junkness":0,
        "preferance":1
    },
    {
        "junkness":2,
        "preferance":0
    },
    {
        "junkness":1,
        "preferance":1
    },
    {
        "junkness":0,
        "preferance":0
    },
    {
        "junkness":2,
        "preferance":1
    }
]
user_names = ['a','b','c','d','e','f']
users = {}
for name,value in zip(user_names,user_list):
  users[name]=value
users

{'a': {'junkness': 1, 'preferance': 0},
 'b': {'junkness': 0, 'preferance': 1},
 'c': {'junkness': 2, 'preferance': 0},
 'd': {'junkness': 1, 'preferance': 1},
 'e': {'junkness': 0, 'preferance': 0},
 'f': {'junkness': 2, 'preferance': 1}}

In [0]:
user_history = {}

In [0]:
user_history['a']= make_user_history(user['a'],3)

In [0]:
for user_name in user_names[1:]:
  user_history[user_name] = list(set(make_user_history(user['a'],3)))

In [0]:
def get_items(recipes):
  recipes = [new_data[new_data.recipe_number == recipe_no ]['items'].values[0] for recipe_no in recipes]
  return [item for recipe in recipes for item in recipe]

In [0]:
def get_intersection(list_2d):
  return list(set(list_2d[0]) & set(list_2d[1]))

In [0]:
def content_based(user_name):
  global user_history,users
  current_user = users[user_name]
  current_user_history = user_history[user_name]
  print(current_user,current_user_history)
  suggestions = []
  likeable_items = []
  for key in classes:
    labels = classes[key]
    for ind,value in enumerate(labels):
      likeable_items.append(get_items(class_data[key][value]))
  intersection_items = get_intersection(likeable_items)
  print(len(intersection_items))
  for elements in intersection_items:
      suggestions.append(elements)
  return suggestions

In [60]:
content_based('a')

['milk', 'sugar', 'salt']


In [0]:
def isalike(u1,u2):
  for key in u1:
    if u1[key] == u2[key]:
      return True
  return False

In [0]:
def get_similiar_users(user_name):
  global users
  current_user_info = users[user_name]
  similiar_users= []
  other_users = [user for user in users.keys() if user!=user_name]
  for other_user_name in other_users:
    user_info = users[other_user_name]
    if isalike(current_user_info,user_info):
      similiar_users.append(other_user_name)
  return similiar_users

In [0]:
def collaborative(user_name):
    global user_history,users
    current_user = users[user_name]
    current_user_history = user_history[user_name]
    similiar_users = get_similiar_users(user_name)
    similiar_histories = []
    for sim_user_name in similiar_users:
      similiar_histories+=user_history[sim_user_name]
    similiar_histories = [i for i in similiar_histories if i not in user_history[user_name]]
    likeable_items = []
    for key in classes:
      labels = classes[key]
      for value in labels:
        likeable_items.append(get_items(class_data[key][value]))
    likeable_items = get_intersection(likeable_items)
    return list(set(likeable_items) & set(similiar_histories))

In [0]:
def get_liked_items(user_name):
  global classes,class_data,users
  user_info = users[user_name]
  for key in classes:
    labels = classes[key]
    likeable_items=[]
    for values in labels:
      likeable_items.append(get_items(class_data[key][values]))
    likeable_items = get_intersection(likeable_items)
    return likeable_items

In [0]:
def collaborative(user_name):
  global users,all_item
  item_score = [0 for _ in len(all_items)]
  other_users = [user for user in users.keys() if user!=user_name]
  for other_name in other_users:
    if isalike(user_name,other_name):
      for item in get_liked_items(other_name):
        item_score[all_items.index(item)]+=1
  return all_item[max(item_score)]